In [1]:
import IPython.display as ipd

import pathlib

import torch
import torchaudio

import textless
from textless.data.speech_encoder import SpeechEncoder
from textless.data.quantized_datasets import QuantizedLibriSpeech
from textless.vocoders.tacotron2.vocoder import TacotronVocoder

In [2]:
dense_model_name = "hubert-base-ls960"
quantizer_name = "kmeans"
vocab_size = 50 # one of [50, 100, 200]

encoder = SpeechEncoder.by_name(
    dense_model_name=dense_model_name,
    quantizer_model_name=quantizer_name,
    vocab_size=vocab_size,
    need_f0=False,
    deduplicate=True,
    f0_normalizer=None,
    f0_quantizer=None,
)#.cuda()

100%|██████████| 1.06G/1.06G [00:10<00:00, 107MB/s] 
/opt/conda/envs/environment-cpu/lib/python3.8/site-packages/fairseq/checkpoint_utils.py:304: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [6]:
! mkdir -p ../datasets

In [7]:
raw_dataset = torchaudio.datasets.LIBRISPEECH(
    root="../datasets",
    url="dev-clean",
    download=True,
)

audio, input_sample_rate, *_ = raw_dataset[7]
audio

100%|██████████| 322M/322M [00:03<00:00, 108MB/s]  


tensor([[-0.0003, -0.0006, -0.0006,  ..., -0.0003, -0.0003, -0.0003]])

In [8]:
ipd.Audio(audio, rate=input_sample_rate)

In [9]:
encoded_audio = encoder(audio)
encoded_audio.keys()

dict_keys(['units', 'durations', 'dense'])

In [10]:
encoded_audio['units'][:10]

tensor([20,  2, 13, 15, 33, 20, 18, 37, 32, 43], dtype=torch.int32)

In [12]:
encoded_audio['dense'][:10].shape # ce sont les outputs du speech-to-unit

torch.Size([10, 768])